In [11]:
from cnnClassifier.utils.common import read_yaml, create_directories

In [1]:
import os 


In [2]:
%pwd

'/Users/davidbedoya0/Documents/Repositorios/Learning/ml-projects/kidney-disease-mlflow-dvc-project/research'

In [3]:
os.chdir("../")

In [ ]:
### ENTITY 

from dataclasses import dataclass
from pathlib import Path

# dataclass is a class that is used to create a data class
# froze = True significa que la clase es inmutable, es decir, no se puede modificar sus atributos
# dataclass es util para crear clases que se usan para almacenar datos ya que no se necesita definir el constructor
@dataclass(frozen=True)
class DataIngestionConfig: 
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [ ]:
### CONFIGURATION MANAGER 

class ConfigurationManager: 
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # se crea el directorio de artefactos si no existe
        create_directories([self.config.artifact_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        # se obtiene la configuracion de la ingestion de datos desde el archivo de configuracion config/config.yaml 
        config = self.config.data_ingestion

        # se crea el directorio de la ingestion de datos si no existe
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os 
import zipfile
import gdown 
from cnnClassifier import logger 
from cnnClassifier.utils.common import get_size


In [ ]:
### COMPONENTS 

class DataIngestion: 
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self)-> str:
        '''
        Fetch data from the url 

        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion" , exist_ok=True)
            logger.info(f"Downloading file from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")

        except Exception as e:
            raise e 
        
    def extract_zip_file(self): 
        '''
        zip_file_path: str 
        Extracts the zip file into the data directory
        Function return None
        '''

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref: 
            zip_ref.extractall(unzip_path)
            


In [12]:
### PIPELINE

try : 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e 


[2025-07-08 20:57:47,222: INFO: common: yaml file: config/config.yaml loaded successfully
[2025-07-08 20:57:47,225: INFO: common: yaml file: params.yaml loaded successfully
[2025-07-08 20:57:47,227: INFO: common: created directory at: artifacts
[2025-07-08 20:57:47,228: INFO: common: created directory at: artifacts/data_ingestion
[2025-07-08 20:57:47,230: INFO: 872948733: Downloading file from https://drive.google.com/file/d/1AZEkvbneTQchqyFqGEaeJ7kwWTgbDDf9/view?usp=share_link into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1AZEkvbneTQchqyFqGEaeJ7kwWTgbDDf9
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1AZEkvbneTQchqyFqGEaeJ7kwWTgbDDf9&confirm=t&uuid=46561894-6ed5-4c6a-8117-979428646f26
To: /Users/davidbedoya0/Documents/Repositorios/Learning/ml-projects/kidney-disease-mlflow-dvc-project/artifacts/data_ingestion/data.zip
100%|██████████| 940M/940M [03:55<00:00, 4.00MB/s] 

[2025-07-08 21:01:44,464: INFO: 872948733: Downloaded data from https://drive.google.com/file/d/1AZEkvbneTQchqyFqGEaeJ7kwWTgbDDf9/view?usp=share_link into artifacts/data_ingestion/data.zip
